In [1]:
!rm -r data/
!rm -r __MACOSX/
!rm README.*

'rm' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!mkdir data
!unzip data.zip -d ./data

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import yaml
import os

# Modify original data label
def update_original_data(yaml_path, labels_paths):
    yaml_data = {}
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
        f.close()

    for path in labels_paths:
        for file in os.listdir(path):
            new_content = []
            # Read label files
            with open(f'{path}/{file}', 'r') as f:
                content = f.readlines()
                for line in content:
                    # update the class id to 0, as only 1 class
                    words = line.split(' ')[1:]
                    new_words = ['0'] + words
                    new_line = ' '.join(new_words)
                    new_content.append(new_line)


            # Write label files
            with open(f'{path}/{file}', 'w') as f:
                f.writelines(new_content)
                f.close()

# Update data/train/labels
labels_paths = ['data/train/labels', 'data/valid/labels']
update_original_data('data/data.yaml', labels_paths)

In [5]:
from os.path import split
import os

# Filter out malformatted files
# For yolo without OBB, each line in label file contains only 5 values
def malformat_filter(paths):
    for path in paths:
        targets = []
        for file in os.listdir(path+'labels'):
            lines = []
            label_path = path+'labels/'+file
            with open(label_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                words = line.split(' ')
                if len(words) > 5:
                    targets.append('.'.join(label_path.split('.')[:-1]))
                    break

        for target in targets:
            os.remove(target+'.txt')
            os.remove(target.replace('labels', 'images', 1)+'.jpg')

malformat_filter(['data/train/', 'data/valid/', 'data/test/'])

In [6]:
# Modify original yaml to accomodate Hong Kong Mahjong Tiles
import yaml
import os

def update_original_yaml(path):
    yaml_content = {}
    with open(path, 'r') as f:
        yaml_content = yaml.safe_load(f)
        print(yaml_content)
        f.close()

    yaml_content['path'] = './data'
    yaml_content['test'] = 'test/images'
    yaml_content['train'] = 'train/images'
    yaml_content['val'] = 'valid/images'
    yaml_content['names'] = ['tile']
    yaml_content['nc'] = len(yaml_content['names'])

    with open('yolo11.yaml', 'w') as f:
        yaml.safe_dump(yaml_content, f)

update_original_yaml('./data/data.yaml')

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 38, 'names': ['0b', '0m', '0p', '0s', '1m', '1p', '1s', '1z', '2m', '2p', '2s', '2z', '3m', '3p', '3s', '3z', '4m', '4p', '4s', '4z', '5m', '5p', '5s', '5z', '6m', '6p', '6s', '6z', '7m', '7p', '7s', '7z', '8m', '8p', '8s', '9m', '9p', '9s'], 'roboflow': {'workspace': 'majmaster', 'project': 'master-oez61', 'version': 7, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/majmaster/master-oez61/dataset/7'}}
